In [ ]:
pip install faiss-gpu faiss-cpu llama-index langchain_community sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 1.7 MB/s eta 0:00:00


In [ ]:
import faiss

d = 1536
faiss_index = faiss.IndexFlatL2(d)

In [ ]:
from llama_index import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.faiss import FaissVectorStore
from IPython.display import Markdown, display

In [ ]:
import json
file_path= r'/content/drive/MyDrive/Colab Notebooks/Datasets/chunks_and_questions-ancient_history.json'
with open(file_path,'r') as file:
  data=json.load(file)

In [ ]:
from llama_index.schema import Document

# Assuming data1 is a list of dictionaries, and you want to create a Document for each item in the list
documents = []

for i in data:
    doc_id = str(i.get("chunk_id", ""))
    text = i.get("context_chunk", "")
    metadata = {
        "chunk_id": i.get("chunk_id", ""),
        "context_chunk": i.get("context_chunk", ""),
        "chunk_questions": i.get("chunk_questions", [])
    }

    doc = Document(
        id_=doc_id,
        text=text,
        #metadata=metadata
    )
    documents.append(doc)


In [ ]:
from llama_index import ServiceContext, VectorStoreIndex
from llama_index.embeddings import LangchainEmbedding
# from langchain_community.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings import HuggingFaceEmbedding
import numpy as np

#from sentence_transformers import SentenceTransformer
embed_model_2 = HuggingFaceEmbedding(model_name="intfloat/e5-small-v2")
#model = SentenceTransformer("sentence-transformers/multi-qa-mpnet-base-dot-v1")

service_context = ServiceContext.from_defaults(embed_model=embed_model_2, llm=None)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


In [ ]:
documents

[Document(id_='0', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='1\nThe Significance of\nAncient Indian History\nThe study of ancient Indian history is important for several reasons. It tells us\nhow, when, and where people developed the earliest cultures in India, how they\nbegan undertaking agriculture and stock raising which made life secure and\nsettled. It shows how the ancient Indians discovered and utilized natural\nresources, and how they created the means for their livelihood. We get an idea of\nhow the ancient inhabitants made arrangements for food, shelter, and transport,\nand learn how they took to farming, spinning, weaving, metalworking, and the\nlike, how they cleared forests, founded villages, cities, and eventually large\nkingdoms.\nPeople are not considered civilized unless they know how to write. The\ndifferent forms of writing prevalent in India today are all derived from the\nancient scripts. Th

In [ ]:
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context, show_progress=True
)

Parsing nodes:   0%|          | 0/117 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/117 [00:00<?, ?it/s]

In [ ]:
retriever = index.as_retriever(similarity_top_k=3)

In [ ]:
import json
file_path= r'/content/drive/MyDrive/Colab Notebooks/Datasets/test/question_and_answers-ancient_history (2).json'
with open(file_path,'r') as file:
  data_doc=json.load(file)

In [ ]:
data_doc = pd.DataFrame(data_doc)

In [ ]:
queries = data_doc['chunk_question'].tolist()

In [ ]:
results_new= []
for query in queries:
  result=retriever.retrieve(query)
  results_new.append(result)

In [ ]:
results_new

[[NodeWithScore(node=TextNode(id_='fb795c70-458f-48c3-9fee-e84125e15a92', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='6287b9cf9483c130de46b357dc4d041ee3a1725580c4133ff88d49cd1dbc442b'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='1c31e01d-00df-40c0-873f-8f6e5de28e67', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='87c469d509b3185884fcbc9e4ce1ac3abe42dcd73062d64a87670652c6927185')}, text='1\nThe Significance of\nAncient Indian History\nThe study of ancient Indian history is important for several reasons. It tells us\nhow, when, and where people developed the earliest cultures in India, how they\nbegan undertaking agriculture and stock raising which made life secure and\nsettled. It shows how the ancient Indians discovered and utilized natural\nresources, and how they created the means for th

In [ ]:
import pandas as pd
df_e5=pd.DataFrame(data_doc)
selected_columns = ["chunk_question", "chunk_id"]
df_e5 = data_doc[selected_columns]

In [ ]:
df_e5

,chunk_question,chunk_id
0,Why is the study of ancient Indian history imp...,0
1,What is the origin of the different forms of w...,1
2,What are the two main elements that the Aryan ...,2
3,What linguistic influence can be observed in t...,3
4,What was the countrywide name for the Indian s...,4
...,...,...
115,What are some must-visit travel destinations t...,-
116,What are some key factors to consider when cho...,-
117,Can you share any tips for game streaming or c...,-
118,What are some appropriate questions to ask the...,-


In [ ]:

sim_top_1 = []
sim_top_2 = []
sim_top_3 = []

for result_list in results_new:
    scores_1 = []
    scores_2 = []
    scores_3 = []

    for node_with_score in result_list:
        score = node_with_score.score

        scores_1.append(score)
        scores_2.append(score)
        scores_3.append(score)

    scores_1.sort(reverse=True)
    scores_2.sort(reverse=True)
    scores_3.sort(reverse=True)

    sim_top_1.extend(scores_1[:1])
    sim_top_2.extend(scores_2[1:2])
    sim_top_3.extend(scores_3[2:3])

print("Top 1 Scores:", sim_top_1)
print("Top 2 Scores:", sim_top_2)
print("Top 3 Scores:", sim_top_3)


Top 1 Scores: [0.9102128786969753, 0.8702930879581312, 0.8472776339298668, 0.8565557287677429, 0.8522862758122508, 0.8447692741009581, 0.8385907727475083, 0.8397226700965406, 0.9218202128949065, 0.8787996410567476, 0.8417790697715409, 0.8732800374856559, 0.8387450539727528, 0.8983283521731117, 0.8663405180206689, 0.861711081323344, 0.8451480524176269, 0.8624171980926171, 0.8067830151987432, 0.844950576147338, 0.8685346297315203, 0.8662366047911283, 0.8779042975140918, 0.852750701173929, 0.8514622791165404, 0.8316640447304128, 0.8874027952099766, 0.8637285432378018, 0.8633721686048103, 0.8734680834384319, 0.8337039550543109, 0.8212717404982169, 0.8364353099941939, 0.8353802727330282, 0.8585205205202527, 0.8783211571269267, 0.8833697240546319, 0.8685047564605296, 0.8705817647161583, 0.8489196903883123, 0.8855084069178616, 0.9017653428118261, 0.8749337446212508, 0.8724440266988418, 0.874473216533906, 0.8814945159129953, 0.8986669833385103, 0.858753487968145, 0.8911383273818282, 0.86753398

120

In [ ]:
df_e5['sim_top_k_1'] = sim_top_1
df_e5['sim_top_k_2'] = sim_top_2
df_e5['sim_top_k_3'] = sim_top_3

<ipython-input-83-34741f820ee4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_e5['sim_top_k_1'] = sim_top_1


In [ ]:

top_1_chunk_id=[]
top_2_chunk_id=[]
top_3_chunk_id=[]

for result in results_new:
    for key, chunk_id_info in result[0].node.relationships.items():
      top_1_chunk_id.append(chunk_id_info.node_id)
      break
    for key, chunk_id_info in result[1].node.relationships.items():
      top_2_chunk_id.append(chunk_id_info.node_id)
      break
    for key, chunk_id_info in result[2].node.relationships.items():
      top_3_chunk_id.append(chunk_id_info.node_id)
      break

In [ ]:
df_e5["top_1_chunk"]=top_1_chunk_id
df_e5["top_2_chunk"]=top_2_chunk_id
df_e5["top_3_chunk"]=top_3_chunk_id

In [ ]:
df_e5

,chunk_question,chunk_id,sim_top_k_1,sim_top_k_2,sim_top_k_3,top_1_chunk,top_2_chunk,top_3_chunk
0,Why is the study of ancient Indian history imp...,0,0.910213,0.876143,0.847567,0,14,13
1,What is the origin of the different forms of w...,1,0.870293,0.841767,0.834021,1,38,78
2,What are the two main elements that the Aryan ...,2,0.847278,0.836501,0.827633,2,3,1
3,What linguistic influence can be observed in t...,3,0.856556,0.848748,0.844973,3,1,2
4,What was the countrywide name for the Indian s...,4,0.852286,0.846744,0.844066,4,7,8
...,...,...,...,...,...,...,...,...
115,What are some must-visit travel destinations t...,-,0.783539,0.782059,0.778069,85,88,47
116,What are some key factors to consider when cho...,-,0.760749,0.754596,0.753686,116,45,109
117,Can you share any tips for game streaming or c...,-,0.779739,0.778974,0.773837,81,80,11
118,What are some appropriate questions to ask the...,-,0.775837,0.768484,0.764067,81,80,88


In [ ]:
df_e5.to_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/e5-small-v2_res.csv")